In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.display import clear_output
from tkinter import Tk, filedialog
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from quick_pp.objects import Project
import quick_pp.las_handler as las
from quick_pp.core_calibration import fit_poroperm_curve, poroperm_xplot, j_xplot, bvw_xplot
from quick_pp.rock_type import rock_typing, rqi, fzi, vsh_gr
from quick_pp.plotter import plotly_log
from quick_pp.core_calibration import sw_shf_leverett_j, sw_shf_cuddy, sw_shf_choo


## Load and merge well data and core data

In [ ]:
# Load well from saved file
project = "M3"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [ ]:
col_rename = {
    'NEUT': 'NPHI',
    'NEUTHC': 'NPHI_HC',
    'DENB': 'RHOB',
    'DENBHC': 'RHOB_HC',
}

In [ ]:
# Load data
well_name = 'M03-2'  # 'M01-3'
well = project.get_well(well_name)
well_data = well.data
well_data = well_data.rename(columns=col_rename)

min_depth = well_data.DEPTH.min()
max_depth = well_data.DEPTH.max()
well_data = well_data[(well_data.DEPTH >= min_depth) & (well_data.DEPTH <= max_depth)].copy()

In [ ]:
core_df = pd.read_csv(r"C:\Users\imran\OneDrive\Desktop\Petrophysics\Data\Core\M1_M3_RAW_SCAL.csv")
core_df = core_df[core_df.WELL_NAME == well_name].sort_values('DEPTH')

In [ ]:
well_data = pd.merge_asof(well_data, core_df[['DEPTH', 'CPORE', 'CPERM']], on='DEPTH',
                          direction='nearest', tolerance=1)

## Rock Typing

In [ ]:
a, b = fit_poroperm_curve(core_df['CPORE'], core_df['CPERM'])
for i, data in core_df.groupby('SAMPLE_NO'):
    poroperm_xplot(data['CPORE'], data['CPERM'], a, b, label=f'RT {i} - a: {a}, b: {b}')

In [ ]:
# # Using FZI
# well_data['FZI'] = fzi(well_data['PERM'], well_data['PHIT'])
# well_data['ROCK_FLAG'] = rock_typing(well_data['FZI'], [.5, .7, 1.3], higher_is_better=True)

# Using VSH_GR
well_data['VSH_GR'] = vsh_gr(well_data['GR'])
well_data['ROCK_FLAG'] = rock_typing(well_data['VSH_GR'], [.07, .12, .2], higher_is_better=False)

fig = plotly_log(well_data)
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))

core_df = pd.merge_asof(core_df.drop(columns='ROCK_FLAG'), well_data[['DEPTH', 'ROCK_FLAG']], on='DEPTH', direction='nearest')

## SHF

In [ ]:
core_df.ROCK_FLAG

In [ ]:
for rock, rock_data in core_df.groupby('SAMPLE_NO'):
    temp_df = pd.DataFrame()
    for i, data in rock_data.iterrows():
        t_df = data[[c for c in rock_data.columns if 'SW' in c]].reset_index(drop=True).T.reset_index(name='SW')
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'PC' in c]].reset_index(drop=True).T.reset_index(name='PC')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'SWN' in c]].reset_index(drop=True).T.reset_index(name='SWN')], axis=1)
        t_df = pd.concat([t_df, data[[c for c in rock_data.columns if 'J' in c]].reset_index(drop=True).T.reset_index(name='J')], axis=1)
        t_df.drop(columns='index', inplace=True)
        temp_df = pd.concat([temp_df, t_df])
    temp_df.dropna(inplace=True)
    temp_df = temp_df[(temp_df > 0) & (temp_df != np.inf)]
    temp_df = temp_df.sort_values(by='J').reset_index(drop=True)
    j_xplot(temp_df['SW'], temp_df['J'], log_log=False,
            label=f"#{rock}")
        # plt.show()

j_constants = {
    'RT1': {'a': 0.02, 'b': -2.5},
    'RT2': {'a': 0.03, 'b': -2.7},
    'RT3': {'a': 0.07, 'b': -3.2},
}
for rock, param in j_constants.items():
    a = param['a']
    b = param['b']
    j_xplot(sw=0, j=0, a=a, b=b, log_log=False, ylim=(0.01, 10),
            label=f"#{rock} - a: {a}, b: {b}")

In [ ]:
for rock, rock_data in core_df.groupby('SAMPLE_NO'):
    temp_df = pd.DataFrame()
    for i, data in rock_data.iterrows():
        cpore = data['CPORE']
        t_df = data[[c for c in rock_data.columns if 'SW' in c]].reset_index(drop=True).T.reset_index(name='SW')
        t_df['BVW'] = t_df['SW'] * cpore
        t_df = pd.concat([t_df, data[[c for c in rock_data.columns if 'PC' in c]].reset_index(drop=True).T.reset_index(name='PC')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'SWN' in c]].reset_index(drop=True).T.reset_index(name='SWN')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'J' in c]].reset_index(drop=True).T.reset_index(name='J')], axis=1)
        t_df.drop(columns='index', inplace=True)
        temp_df = pd.concat([temp_df, t_df])
    temp_df.dropna(inplace=True)
    temp_df = temp_df[(temp_df > 0) & (temp_df != np.inf)]
    temp_df = temp_df.sort_values(by='PC').reset_index(drop=True)
    bvw_xplot(temp_df['BVW'], temp_df['PC'], label=f"RT {rock}")

cuddy_constants = {
    1: {'a': 0.01, 'b': -2.0},
    2: {'a': 0.02, 'b': -2.3},
    3: {'a': 0.03, 'b': -3.2},
}
for rock, param in cuddy_constants.items():
    a = param['a']
    b = param['b']
    bvw_xplot(0, 0, a, b, label=f"RT {rock} - a: {a}, b: {b}", ylim=(0.01, 200), log_log=True)

In [ ]:
FWL = 6000  # 5967

j_constants = {
    1: {'a': 0.02, 'b': -2.5},
    2: {'a': 0.03, 'b': -2.7},
    3: {'a': 0.07, 'b': -3.2},
}
cuddy_constants = {
    1: {'a': 0.01, 'b': -2.0},
    2: {'a': 0.02, 'b': -2.3},
    3: {'a': 0.03, 'b': -3.2},
}
for rock, rock_data in well_data.groupby('ROCK_FLAG'):
    if rock == 4:
         well_data.loc[well_data['ROCK_FLAG'] == rock, ['SHF_J', 'SHF_Choo', 'SHF_Cuddy']] = 1
    else:
        a = j_constants[rock]['a']
        b = j_constants[rock]['b']
        well_data.loc[well_data['ROCK_FLAG'] == rock, 'SHF_J'] = sw_shf_leverett_j(
            rock_data['PERM'], rock_data['PHIT'], rock_data['DEPTH'], 
            fwl=FWL, ift=26, gw=0.433, ghc=0.35, a=a, b=b
        )

        well_data.loc[well_data['ROCK_FLAG'] == rock, 'SHF_Choo'] = sw_shf_choo(
            well_data['PERM'], well_data['PHIT'], well_data['PHIE'], well_data['DEPTH'],
            fwl=FWL, ift=26, gw=0.433, ghc=0.35, b0=0.4
        )

        a = cuddy_constants[rock]['a']
        b = cuddy_constants[rock]['b']
        well_data.loc[well_data['ROCK_FLAG'] == rock, 'SHF_Cuddy'] = sw_shf_cuddy(
            well_data['PHIT'], well_data['DEPTH'],
            fwl=FWL, gw=0.433, ghc=0.35, a=0.27, b=-2.0124)

well_data[['DEPTH', 'SWT', 'SHF_J', 'SHF_Choo', 'SHF_Cuddy']].plot(x='DEPTH', figsize=(10, 3))
plt.ylim(0, 1.1)
plt.xlim(5600, 6200)

In [ ]:
well_data['SHF'] = well_data['SHF_Cuddy']

fig = plotly_log(well_data)
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))

In [ ]:
# from quick_pp.core_calibration import estimate_hafwl

# hafwl = estimate_hafwl(
#     sw=well_df['SW[0]'],
#     poro=well_df['CPORE'],
#     perm=well_df['CPERM'],
#     ift=26,
#     gw=1.0,
#     ghc=0.7,
#     a=0.507,
#     b=2.455
# )
# hafwl

In [ ]:
# # Need HAFWL to plot 
# df.plot(x='BVW', y='DEPTH_SHIFTED', kind='scatter', logy=False, xlim=(0, 0.5))

In [ ]:
# df[[c for c in df.columns if 'PC' in c]].describe()

In [ ]:
# for i in range(8):
#     df.plot(x=f'SW[{i}]', y=f'PC[{i}]', kind='scatter', logy=False, xlim=(0, 1))
# plt.show()